# The Evaluation Pipeline

In Notebook 2 we built case files and ran one case through the agent by hand. This notebook automates that process across the entire dataset and attaches structured graders at three levels: per-item, per-trace, and per-run.

This notebook covers:
1. Uploading the case dataset to Langfuse
2. What each grader measures and why
3. Running the full experiment
4. Inspecting results in Python and in the Langfuse UI

---

**Prerequisites:** Complete Notebooks 1 and 2. The case file must exist at `implementations/aml_investigation/data/aml_cases.jsonl`.

In [ ]:
import os
from pathlib import Path

import pandas as pd
from aieng.agent_evals.aml_investigation.graders.item import item_level_deterministic_grader
from aieng.agent_evals.aml_investigation.graders.run import run_level_grader
from aieng.agent_evals.aml_investigation.graders.trace import trace_deterministic_grader
from aieng.agent_evals.aml_investigation.task import AmlInvestigationTask
from aieng.agent_evals.evaluation import run_experiment_with_trace_evals
from aieng.agent_evals.langfuse import upload_dataset_to_langfuse
from dotenv import load_dotenv


# Setting the notebook directory to the project's root folder
if Path("").absolute().name == "eval-agents":
    print(f"Notebook path is already the root path: {Path('').absolute()}")
else:
    os.chdir(Path("").absolute().parent.parent)
    print(f"The notebook path has been set to: {Path('').absolute()}")

CASES_PATH = Path("implementations/aml_investigation/data/aml_cases.jsonl")
DATASET_NAME = "aml-investigation-eval"

assert CASES_PATH.exists(), f"Cases file not found at {CASES_PATH}. Run Notebook 2 first."
print(f"Found {sum(1 for line in CASES_PATH.read_text().splitlines() if line.strip())} cases.")

load_dotenv(verbose=True)

## 1. Uploading the Dataset to Langfuse

Langfuse acts as the backbone of our evaluation pipeline. Each case file becomes a dataset item in Langfuse: the `input` field (the `CaseFile`) is what gets sent to the agent, and the `expected_output` field (the `GroundTruth`) is stored separately and made available to the graders.

`upload_dataset_to_langfuse` reads the JSONL file, creates the dataset if it does not already exist, and upserts items using a deterministic content-based ID. Running this cell twice is safe: existing items are updated in place rather than duplicated.

In [ ]:
await upload_dataset_to_langfuse(dataset_path=str(CASES_PATH), dataset_name=DATASET_NAME)

print(f"Dataset '{DATASET_NAME}' is ready in Langfuse.")

## 2. The Graders

We use three layers of graders. Each layer answers a different question about the agent.

### 2.1 Item-level graders

Item-level graders run once per case and compare the agent's `AnalystOutput` to the `GroundTruth`. They are deterministic: no LLM judge is involved.

| Metric | What it measures |
|---|---|
| `is_laundering_correct` | Whether the agent's verdict (laundering or not) matches ground truth |
| `pattern_type_correct` | Whether the agent named the exact correct pattern (e.g. `FAN-OUT`, `NONE`) |
| `non_laundering_pattern_consistent` | When the agent predicts benign, whether the pattern is `NONE` |
| `non_laundering_flags_empty` | When the agent predicts benign, whether no transaction IDs are flagged |
| `id_precision_like` | Of the flagged IDs, how many were correct minus how many were wrong, normalized by count |
| `id_coverage` | Of the ground truth laundering IDs, what fraction the agent correctly identified |

The two consistency checks (`non_laundering_pattern_consistent` and `non_laundering_flags_empty`) probe a specific failure mode: an agent that correctly says "not laundering" but still outputs a suspicious-looking pattern name or a non-empty list of flagged IDs. These outputs would confuse a downstream consumer even if the verdict is right.

### 2.2 Trace-level graders

Trace-level graders inspect the agent's SQL tool calls from the Langfuse trace, not just the final output. They run in a second pass after the experiment finishes, once all traces have been ingested.

| Metric | What it measures |
|---|---|
| `trace_has_sql_queries` | Whether the agent issued at least one database query |
| `trace_read_only_query_check` | Whether all queries were read-only (no `INSERT`, `UPDATE`, `DROP`, etc.) |
| `trace_window_filter_present` | Whether at least one query referenced the case investigation window |
| `trace_window_violation_count` | How many queries used timestamps outside the case investigation window |
| `trace_redundant_query_ratio` | Fraction of queries that were exact duplicates of a previous query in the same run |

These metrics catch issues that would be invisible from the final output alone. An agent could produce a perfect verdict by hallucinating rather than querying, issue write queries, look at data outside the permitted window, or waste its context budget on redundant queries.

### 2.3 Run-level graders

Run-level graders receive all item results after the experiment finishes and compute aggregate classification metrics across the full dataset.

| Metric | What it measures |
|---|---|
| `is_laundering_precision` | Precision for laundering detection across all cases |
| `is_laundering_recall` | Recall for laundering detection across all cases |
| `is_laundering_f1` | F1 score for laundering detection |
| `pattern_type_macro_f1` | Macro-averaged F1 across all pattern types |
| `pattern_type_confusion_matrix` | Full confusion matrix over pattern types (stored in metadata) |

## 3. Running the Experiment

The experiment runs in two passes.

Pass 1 executes the task over every dataset item up to `max_concurrency` cases at a time. For each item, the `AmlInvestigationTask` sends the `CaseFile` JSON to the agent, streams its response, and parses it into an `AnalystOutput`. The item-level graders score immediately after each item finishes, while the run-level graders wait until all items are done and then compute aggregate metrics.

Pass 2 waits for all traces to be fully ingested by Langfuse, then runs the trace-level graders. This second pass is necessary because trace data arrives asynchronously: the agent may finish producing output before all intermediate tool-call spans have been written to Langfuse.

> **Note:** This cell makes live LLM calls for every case in the dataset. With 16 cases and `max_concurrency=4`, expect 10 to 15 minutes total depending on model latency.

In [ ]:
task = AmlInvestigationTask()

result = run_experiment_with_trace_evals(
    DATASET_NAME,
    name="aml-investigation-baseline",
    task=task,
    evaluators=[item_level_deterministic_grader],
    trace_evaluators=[trace_deterministic_grader],
    run_evaluators=[run_level_grader],
    description="Baseline AML investigation agent evaluation.",
    max_concurrency=4,
)

print("Experiment complete.")

In [ ]:
await task.close()
print("Task closed.")

## 4. Inspecting Results

The `result` object gives programmatic access to every item-level score and to the trace evaluations. Run-level aggregate metrics are computed by the `run_level_grader` and pushed to the Langfuse experiment run; the most convenient place to read them is the Langfuse UI.

### 4.1 Item-level scores

In [ ]:
rows = []
for item_result in result.experiment.item_results:
    item = item_result.item
    case_input = item.get("input") if isinstance(item, dict) else item.input
    expected = item.get("expected_output") if isinstance(item, dict) else item.expected_output

    row = {
        "case_id": case_input.get("case_id", "")[:12] + "...",
        "trigger_label": case_input.get("trigger_label", ""),
        "gt_laundering": expected.get("is_laundering"),
        "gt_pattern": expected.get("pattern_type"),
    }

    if item_result.output:
        output = item_result.output
        row["pred_laundering"] = output.get("is_laundering")
        row["pred_pattern"] = output.get("pattern_type").value
    else:
        row["pred_laundering"] = None
        row["pred_pattern"] = None

    for evaluation in item_result.evaluations or []:
        row[evaluation.name] = round(evaluation.value, 3) if evaluation.value is not None else None

    rows.append(row)

item_df = pd.DataFrame(rows)
print(item_df)

In [ ]:
# Mean score per metric across all cases
score_cols = [
    "is_laundering_correct",
    "pattern_type_correct",
    "non_laundering_pattern_consistent",
    "non_laundering_flags_empty",
    "id_precision_like",
    "id_coverage",
]
available = [c for c in score_cols if c in item_df.columns]
item_df[available].mean().rename("mean").to_frame()

### 4.2 Scores by case type

Breaking down `is_laundering_correct` by case type shows where the agent struggles. True positives and true negatives are the easiest; false positives and false negatives are where most agents lose points.

In [ ]:
_LOW_SIGNAL = {"QA_SAMPLE", "RANDOM_REVIEW", "RETROSPECTIVE_REVIEW", "MODEL_MONITORING_SAMPLE"}
_HIGH_SIGNAL = {"ANOMALOUS_BEHAVIOR_ALERT", "LAW_ENFORCEMENT_REFERRAL", "EXTERNAL_TIP"}
_PATTERN_LABELS = {
    "FAN-IN",
    "FAN-OUT",
    "CYCLE",
    "GATHER-SCATTER",
    "SCATTER-GATHER",
    "STACK",
    "RANDOM",
    "BIPARTITE",
}


def classify_row(row):
    """Classify a case based on its trigger label and laundering status."""
    label = row["trigger_label"]
    is_laundering = row["gt_laundering"]
    if label in _PATTERN_LABELS and is_laundering:
        return "True Positive"
    if label in _LOW_SIGNAL and not is_laundering:
        return "True Negative"
    if (label in _HIGH_SIGNAL or label in _PATTERN_LABELS) and not is_laundering:
        return "False Positive"
    if label in _LOW_SIGNAL and is_laundering:
        return "False Negative"
    return "Other"


item_df["case_type"] = item_df.apply(classify_row, axis=1)

if "is_laundering_correct" in item_df.columns:
    breakdown = (
        item_df.groupby("case_type")[["is_laundering_correct", "pattern_type_correct", "id_coverage"]].mean().round(3)
    )
    print(breakdown.to_string())

### 4.3 Trace-level scores

In [ ]:
if result.trace_evaluations:
    trace_rows = []
    for trace_id, evaluations in result.trace_evaluations.evaluations_by_trace_id.items():
        row = {"trace_id": trace_id[:12] + "..."}
        for evaluation in evaluations or []:
            row[evaluation.name] = round(evaluation.value, 3) if evaluation.value is not None else None
        trace_rows.append(row)

    trace_df = pd.DataFrame(trace_rows)
    print(trace_df.to_string(index=False))
else:
    print("No trace evaluations available.")

In [ ]:
# Mean trace scores across all cases
trace_score_cols = [
    "trace_has_sql_queries",
    "trace_read_only_query_check",
    "trace_window_filter_present",
    "trace_window_violation_count",
    "trace_redundant_query_ratio",
]
if result.trace_evaluations:
    available_trace = [c for c in trace_score_cols if c in trace_df.columns]
    print(trace_df[available_trace].mean().rename("mean").to_frame())

### 4.4 Run-level aggregate metrics

The run-level grader computes precision, recall, F1 for laundering detection and macro F1 for pattern classification across all cases. These are uploaded to Langfuse and visible in the experiment run summary. You can also extract them from the `ExperimentResult` object returned by the first pass.

In [ ]:
print(f"{'Metric':<40} {'Value'}")
print("-" * 50)
for evaluation in result.experiment.run_evaluations:
    if evaluation.name != "pattern_type_confusion_matrix":  # shown separately below
        print(f"{evaluation.name:<40} {evaluation.value:.3f}")

In [ ]:
# Pattern confusion matrix
cm_eval = next((e for e in result.experiment.run_evaluations if e.name == "pattern_type_confusion_matrix"), None)
if cm_eval and cm_eval.metadata:
    labels = cm_eval.metadata.get("labels", [])
    matrix = cm_eval.metadata.get("matrix", [])
    if labels and matrix:
        cm_df = pd.DataFrame(matrix, index=labels, columns=labels)
        cm_df.index.name = "actual \\ predicted"
        print(cm_df.to_string())

## 5. Exploring Traces in the Langfuse UI

The metrics above tell you _what_ the agent got right or wrong, but not _why_. To understand reasoning failures, the Langfuse UI is the right tool. Each experiment item links to the full trace, where you can see every SQL query the agent issued, the model's intermediate reasoning steps, and where in the investigation it went off track.

A few things worth looking at in the UI:

- **False negative cases** where `is_laundering_correct = 0`: did the agent query the database at all? Did it query the right accounts? Did it stop too early?
- **False positive cases** where `is_laundering_correct = 0`: did the agent over-weight the `trigger_label`? Did it find a real pattern or hallucinate one?
- **Cases with high `trace_redundant_query_ratio`**: the agent is burning context on repeated queries, which may indicate the prompt strategy for query budgeting is not working.
- **Cases with `trace_window_violation_count > 0`**: the agent is looking at transactions outside the permitted window, which would be a compliance issue in a real deployment.

## 6. Iterating on the Agent

The evaluation pipeline is designed to make iteration straightforward. The dataset in Langfuse is persistent: once uploaded, you do not need to re-upload it. To evaluate a modified agent, call `run_experiment_with_trace_evals` again with a new `name` argument. Langfuse will create a new experiment run and you can compare runs side by side in the UI.

Common levers to explore:

- **System prompt changes**: edit `ANALYST_PROMPT` in `agent.py` and re-run the experiment. For example, adjust the query strategy section to reduce redundant queries, or revise the typology descriptions to improve pattern classification.
- **Temperature and sampling**: pass `temperature=0.0` to `create_aml_investigation_agent` for more deterministic outputs, or increase it to probe sensitivity.
- **Lookback window**: change `lookback_days` when generating cases to make investigation windows wider or narrower and see how the agent adapts.
- **Case mix**: adjust the ratio of true positives, false positives, and false negatives in `build_cases` to stress-test specific failure modes.